In [51]:
import pandas as pd
from collections import defaultdict
import numpy as np

In [52]:
def main(csv_file):
    """
    Given an appropriate csv_file, output the relevant columns.

    Returns df with columns [user, start, end, normalized_company, industry]
    """

    employ_data = pd.read_csv(csv_file, sep="\t", header=None,
                              names=[i for i in range(34)], low_memory=False)
    # column info from taxonomy file
    name = ['user', 'name', 'birth', 'gender', 'primary',
            'primary_weight', 'secondary', 'secondary_weight',
            'city', 'country', 'education', 'elite', 'start',
            '.', 'end', '??', '/', 'length', 'role', 'department',
            'company', 'normalized_company', 'ticker', 'exchange',
            'public', 'location_company', 'industry', 'educational',
            'degree', 'elite_education', 'major', 'department', 'FIGI',
            'last_update']
#     drop = ['length', 'gender', 'primary',
#         'primary_weight', 'secondary', 'secondary_weight',
#         'city', 'country', 'education', 'elite', '.', '??',
#         '/', 'department', 'exchange',
#         'public', 'location_company', 'educational', 'degree', 'elite_education',
#         'major', 'department', 'FIGI', 'last_update']
    employ_data.columns = name
    return employ_data


# data without datetime features, and none values for some dates
raw_data = {'db': main('./Data/DB_profiles.csv'),
            'gs': main('./Data/GS_profiles.csv'),
            'leh': main('./Data/LEH_profiles.csv'),
            'ms': main('./Data/MS_profiles.csv'),
            'ubs': main('./Data/UBS_profiles.csv')
            }

In [53]:
def standardize_dates(company, missing_start = '1900-01-01', missing_end = '2018-01-01'):
    """
    Converts start date and end date to datetime objects, and converts None values to the specified missing 
    dates.

    Returns the modified dataframe
    """
    company_data = raw_data[company].copy()
    company_data['start'] = company_data['start'].str.replace('None', missing_start)
    company_data['end'] = company_data['end'].str.replace('None', missing_end)
    company_data['start'] = pd.to_datetime(company_data['start'])
    company_data['end'] = pd.to_datetime(company_data['end'])
    return company_data


# set up dictionary to hold data for each company

for company in raw_data.keys():
    raw_data[company] = standardize_dates(company)

In [54]:
def get_users(company_name, company_data, worked_date = '2008-01-01', missing_start = '1900-01-01', missing_end = '2018-01-01'):
    """"
    Returns the users who worked at a given company on worked_date, that does not have both start and
    end dates missing
    
    worked_date: string specifying the date on which to extract employees from. 
                 Must be coercible into a datetime object
    missing_start: default value for missing start dates
    missing_end: default value for missing end dates
    """
    worked_date = pd.to_datetime(worked_date)
    missing_start = pd.to_datetime(missing_start)
    missing_end = pd.to_datetime(missing_end)
    x = company_data
    
    company_tickers = {'db': 'DB', 'leh': 'LEH', 'gs': 'GS', 'ms': 'MS^E', 'ubs': 'UBS'}
    # conditions: start and end not both missing, worked before/after 2008-01-01, ticker matches company
    mask = ~((x['start'] == missing_start) & (x['end'] == missing_end)) & \
        (x['start'] < worked_date) & \
        (x['end'] > worked_date) & \
        (x['ticker'] == company_tickers[company_name])
    return company_data[mask]['user'].unique()


# gets the user_ids within each company that match the conditioning, before and and after
#   2008-01-01, exclusive
users = {company_name: get_users(company_name, company_data) for company_name, company_data in raw_data.items()}

In [55]:
data = {}
for company, company_data in raw_data.items():
    company_users = users[company]
    data[company] = company_data[company_data['user'].isin(company_users)]

# Missing industries

In [56]:
# read all the csv files
profile = pd.read_csv('./Data/profile_industry_mappings.csv', header=None, names=[i for i in range(5)], dtype={4: str})
profile.drop([0, 2], axis='columns', inplace=True)
profile.rename(mapper={1: 'company', 3: 'norm', 4: "ind"}, axis='columns', inplace=True)

mturk = pd.read_csv('./Data/industries_MTurkers_20170711.csv', header=None, encoding='latin-1')
mturk.drop([1], axis='columns', inplace=True)
mturk.rename(mapper={0: 'company', 2: "ind"}, axis='columns', inplace=True)

finance = pd.read_csv('./Data/Finance.csv', dtype={'Industry': str})
finance.drop([finance.columns[0], finance.columns[2], finance.columns[4]], axis='columns', inplace=True)
finance.rename(mapper={'Normalized Company Name': 'norm', 'Industry': "ind"}, axis='columns', inplace=True)

manual = pd.read_csv('./Data/manual_industry_mappings.csv', encoding='latin-1', header=None, dtype={2: str})
manual.drop([1], axis='columns', inplace=True)
manual.rename(mapper={0: 'norm', 2: "ind"}, axis='columns', inplace=True)

industries_2019 = pd.read_csv('./Data/missing_industries_2019.csv', header=None, dtype={2: str})
industries_2019 = industries_2019[~(industries_2019[1] == 1)].copy()

industries_2019.drop([1], axis = 'columns', inplace = True)
industries_2019.rename(mapper={0: 'company', 2: "ind"}, axis='columns', inplace=True)
industries_2019 = industries_2019[~pd.isnull(industries_2019.ind)].copy()

#mturk industry is given as "ind_x", profile industry is given as "ind_y"
company_comb = pd.merge(mturk, profile, on='company', how='outer')
#prioritize mturk data
company_comb['combined'] = company_comb['ind_x'].combine_first(company_comb['ind_y'])

#mturk industry is given as "ind", profile industry is given as "combined"
company_comb = pd.merge(industries_2019, company_comb, on='company', how='outer')
#prioritize manual entry data
company_comb['combined'] = company_comb['ind'].combine_first(company_comb['combined'])

#merge manual and finance files, prioritizing manual
norm_comb = pd.merge(manual, finance, on = 'norm', how = 'outer')
norm_comb['combined'] = norm_comb['ind_x'].combine_first(norm_comb['ind_y'])
#merge manual/finance and profile[norm], prioritizing manual/finance
norm_comb = pd.merge(norm_comb, profile, on = 'norm', how = 'outer')
norm_comb['combined'] = norm_comb['combined'].combine_first(norm_comb['ind'])

# convert the columns of the aggredated dataframe into a dictionary where the key is the company name
# and the value is the industry code
norm_mapping = dict(zip(norm_comb.norm, norm_comb.combined))
company_mapping = dict(zip(company_comb.company, company_comb.combined))
# set the default value if the company is not found to NaN
norm_mapping = defaultdict(lambda: np.NaN, norm_mapping)
company_mapping = defaultdict(lambda: np.NaN, company_mapping)

def filter_manual(company_data):
    """
    Adds industry labels to entries that don't have one, based on the manual industry data
    """
    company_data = company_data.copy()
    # convert to lowercase for more accurate matching
    company_data['normalized_company_lower'] = company_data['normalized_company'].str.lower()
    company_data['company_lower'] = company_data['company'].str.lower()
    # apply norm_mapping and company_mapping to upper and lower case versions
    company_data['company_mapped'] = company_data['company'].apply(lambda y: company_mapping[y])
    company_data['normalized_company_mapped'] = company_data['normalized_company'].apply(lambda y: norm_mapping[y])
    company_data['company_lower_mapped'] = company_data['normalized_company_lower'].apply(lambda y: norm_mapping[y])
    company_data['normalized_company_lower_mapped'] = company_data['normalized_company_lower'].apply(lambda y: norm_mapping[y])
    # combines all mappings. Prioritize Existing industry code > MTurk/profle(company) > 
    # manual/finance/profile(normalized_company) > manual/finance/profile(normalized_company_lower) 
    company_data['industry_two'] = company_data['industry'].combine_first(company_data['company_mapped'])
    company_data['industry_three'] = company_data['industry_two'].combine_first(company_data['normalized_company_mapped'])
    company_data['industry_four'] = company_data['industry_three'].combine_first(company_data['company_lower_mapped'])
    company_data['industry_five'] = company_data['industry_four'].combine_first(company_data['normalized_company_lower_mapped'])
    company_data['industry'] = company_data['industry_five']
    # drop the temporary columns
    company_data.drop(['normalized_company_lower', 'company_lower', 'company_mapped', 'normalized_company_mapped', 'company_lower_mapped','normalized_company_lower_mapped', 'industry_two', 'industry_three', 'industry_four','industry_five'], axis=1, inplace=True)
    return company_data

In [57]:
def mask(company_data):
    """
    Return values in the time range with start before '2016-1-1' and end after '2016-1-1'.

    Excludes values that don't have a start or end time, or are educational.
    """
    mask = (company_data['start'] <= pd.to_datetime('2016-1-1')) & (company_data['end'] >= pd.to_datetime('2016-1-1')) & ~((company_data['start'] == pd.to_datetime('1900-01-01')) & (company_data['end'] == pd.to_datetime('2018-01-01'))) & (~company_data['ticker'].isin(['UNIVERSITY', 'SCHOOL']) & ~(company_data.educational))
    #& ~(pd.isnull(company_data.industry))
    return company_data[mask]


def filter_and_mask(company_data):
    # combines filter and mask
    filtered = filter_manual(company_data)
    return mask(filtered)

# Missing Industry

In [58]:
# def mask_missing(company_data):
#     """
#     Return values in the time range with start before '2016-1-1' and end after '2016-1-1'.

#     Excludes values that don't have a start or end time.
#     """
    
#     mask = (company_data['start'] < pd.to_datetime('2016-1-1')) & (company_data['end'] > pd.to_datetime('2016-1-1')) & (company_data['start'] != pd.to_datetime('1900-01-01')) & (~company_data['ticker'].isin(['UNIVERSITY', 'TIME_OFF', 'SCHOOL', 'MISSING', 'FREELANCE', 'UNEMPLOYED', 'RETIRED']))
#     return company_data[mask]

# all_data =  pd.concat(raw_data.values())

# company_data = mask_missing(all_data).copy()

# company_data['normalized_company_lower'] = company_data['normalized_company'].str.lower()
# company_data['company_lower'] = company_data['company'].str.lower()
# # apply norm_mapping and company_mapping to upper and lower case versions
# company_data['company_mapped'] = company_data['company'].apply(lambda y: company_mapping[y])
# company_data['normalized_company_mapped'] = company_data['normalized_company'].apply(lambda y: norm_mapping[y])
# company_data['company_lower_mapped'] = company_data['normalized_company_lower'].apply(lambda y: norm_mapping[y])
# company_data['normalized_company_lower_mapped'] = company_data['normalized_company_lower'].apply(lambda y: norm_mapping[y])
# # combines all mappings. Prioritize Existing industry code > MTurk/profle(company) > 
# # manual/finance/profile(normalized_company) > manual/finance/profile(normalized_company_lower) 
# company_data['industry_two'] = company_data['industry'].combine_first(company_data['company_mapped'])
# company_data['industry_three'] = company_data['industry_two'].combine_first(company_data['normalized_company_mapped'])
# company_data['industry_four'] = company_data['industry_three'].combine_first(company_data['company_lower_mapped'])
# company_data['industry_five'] = company_data['industry_four'].combine_first(company_data['normalized_company_lower_mapped'])
# company_data['industry'] = company_data['industry_five']
# # drop the temporary columns
# company_data.drop(['company_mapped', 'normalized_company_mapped', 'company_lower_mapped','normalized_company_lower_mapped', 'industry_two', 'industry_three', 'industry_four','industry_five'], axis=1, inplace=True)

# x = company_data.groupby('user').last()
# most_recent_missing = x[pd.isnull(x['industry'])]

#most_recent_missing['company'].value_counts().to_csv('./Deliverables/missing_values.csv')

# Matching with missing job entries as of 2016-1-1

In [59]:
# begin matching on job titles, prepare data by dropping irrelevant names
drop = ['length', 'gender', 'primary',
        'primary_weight', 'secondary', 'secondary_weight',
        'city', 'country', 'education', 'elite', '.', '??',
        '/', 'department', 'exchange',
        'public', 'location_company', 'educational', 'degree', 'elite_education',
        'major', 'department', 'FIGI', 'last_update', 'industry', 'birth', 'company']
matching_data = {company_name: company_data.drop(labels=drop, axis=1) for company_name, company_data in data.items()}


def job_2008(company_name, company_data):
    """"
    Return each user's job at the given company as of 2008-01-01
    """
    date_2008 = pd.to_datetime('2008-01-01')
    missing_start = pd.to_datetime('1900-01-01')
    missing_end = pd.to_datetime('2018-01-01')

    company_tickers = {'db': 'DB', 'leh': 'LEH', 'gs': 'GS', 'ms': 'MS^E', 'ubs': 'UBS'}

    x = company_data
    mask = ~((x['start'] == missing_start) & (x['end'] == missing_end)) & \
           (x['start'] < date_2008) & \
           (x['end'] > date_2008) & \
           (x['ticker'] == company_tickers[company_name])
    return company_data[mask]


job_as_of_2008 = {company_name: job_2008(company_name, company_data) for company_name, company_data in
                  matching_data.items()}

In [60]:
all_data = pd.concat(job_as_of_2008.values())
# only person missing a role in the entire data set
all_data = all_data.drop(11512)

# begin extracting job titles
directors = set(all_data[(all_data.role.str.contains(r'director|MD,md', case=False))
                         | (all_data.role.str.match(r'ed|md', case=False))].user)
all_roles = directors.copy()

analysts = set(all_data[all_data.role.str.contains('analyst|Anaylst', case=False)].user).difference(all_roles)
all_roles = all_roles.union(analysts)

vps = set(all_data[all_data.role.str.contains('president|vp', case=False)].user).difference(all_roles)
all_roles = all_roles.union(vps)

assocs = set(all_data[all_data.role.str.contains('associate', case=False)].user).difference(all_roles)
all_roles = all_roles.union(assocs)

accountants = set(
    all_data[all_data.role.str.contains('accountant|account executive|accounting', case=False)].user).difference(
    all_roles)
all_roles = all_roles.union(accountants)

consultants = set(all_data[all_data.role.str.contains('consultant', case=False)].user).difference(all_roles)
all_roles = all_roles.union(consultants)

missing = set(all_data[all_data.role.str.match(r'-|\?|\.', case=False)].user).difference(all_roles)
all_roles = all_roles.union(missing)

developers = set(
    all_data[all_data.role.str.contains(r'developer|engineer|system administrator', case=False)].user).difference(
    all_roles)
all_roles = all_roles.union(developers)

interns = set(all_data[all_data.role.str.contains('intern|trainee|apprentice', case=False)].user).difference(all_roles)
all_roles = all_roles.union(interns)

specialists = set(
    all_data[all_data.role.str.contains('specialist|administrator|research|expert', case=False)].user).difference(
    all_roles)
all_roles = all_roles.union(specialists)

sales = set(all_data[all_data.role.str.contains('sales', case=False)].user).difference(all_roles)
all_roles = all_roles.union(sales)

traders = set(all_data[all_data.role.str.contains(r'trader|trading|Portfolio Management', case=False)].user).difference(
    all_roles)
all_roles = all_roles.union(traders)

bankers = set(all_data[all_data.role.str.contains(r'banking|banker|finance', case=False)].user).difference(all_roles)
all_roles = all_roles.union(bankers)

controllers = set(all_data[all_data.role.str.contains('controller', case=False)].user).difference(all_roles)
all_roles = all_roles.union(controllers)

partners = set(all_data[all_data.role.str.contains('partner', case=False)].user).difference(all_roles)
all_roles = all_roles.union(partners)

counsels = set(all_data[all_data.role.str.contains('counsel', case=False)].user).difference(all_roles)
all_roles = all_roles.union(counsels)

recruiters = set(all_data[all_data.role.str.contains('recruiter|human resources', case=False)].user).difference(
    all_roles)
all_roles = all_roles.union(recruiters)

advisors = set(all_data[all_data.role.str.contains('advisor|adviseur', case=False)].user).difference(all_roles)
all_roles = all_roles.union(advisors)

assistants = set(
    all_data[all_data.role.str.contains('assistant|support|services|receptionist', case=False)].user).difference(
    all_roles)
all_roles = all_roles.union(assistants)

managers = set(all_data[all_data.role.str.contains(
    r'manager|supervisor|team lead|head|lead|coordinator|representative|process executive',
    case=False)].user).difference(all_roles)
all_roles = all_roles.union(managers)

others = set(all_data.user).difference(all_roles)

# zip all sets and all job title names
all_sets = [directors, analysts, vps, assocs, advisors, assistants, consultants, managers, missing, developers, interns,
            specialists, sales, traders, bankers, controllers, partners, counsels, recruiters, accountants, others]
job_titles = ['director', 'analyst', 'vp', 'assoc', 'advisor', 'assistant', 'consultant', 'manager', 'missing',
              'developer', 'intern', 'specialist', 'sale', 'trader', 'banker', 'controller', 'partner', 'counsel',
              'recruiter', 'accountant', 'other']

zipped = list(zip(all_sets, job_titles))


def to_dict(dictionary, users, job_title):
    """Map users to job_title in the given dictionary"""
    for user in users:
        dictionary.update({user: job_title})


full_mapping = {}
[to_dict(full_mapping, x, y) for x, y in zipped]
full_mapping.update({'c0a3eb6a-59db-3a30-8a39-99a7cc8b9ce1': 'specialist'})
full_mapping.update({'5f425323-1cdf-3e81-a08e-35b483c42da9': 'missing'})

# Logistic Regression

In [61]:
import statsmodels.discrete.discrete_model as sm
# prepare data for regression by dropping irrelevant names
drop = ['length', 'name',
        'primary_weight', 'secondary', 'secondary_weight', 'elite_education',
        'city', 'country', '.', '??',
        '/', 'department', 'exchange',
        'public', 'location_company',
        'major', 'department', 'FIGI', 'last_update', 'degree']

regression_data = {company_name: company_data.drop(labels=drop, axis=1) for company_name, company_data in data.items()}

In [62]:
#additional step of filtering out those who don't have a job entry on 2016-1-1
all_data_2016 = pd.concat(regression_data.values())
employed_2016 = filter_and_mask(all_data_2016)
employ_2016_users = list(employed_2016.user.unique())

In [63]:
regression_data = {company_name: job_2008(company_name, company_data) for company_name, company_data in
                   regression_data.items()}

In [64]:
non_lehman = pd.concat([regression_data['db'], regression_data['gs'], regression_data['ms'], regression_data['ubs']])
non_lehman['is_lehman'] = 0

lehman = regression_data['leh'].copy()
lehman['is_lehman'] = 1

all_data = pd.concat([lehman, non_lehman])

all_data = all_data[all_data.user.isin(employ_2016_users)]

# fill in missing births to the median date, 1976
index = all_data[all_data.birth.isin(['None', '2000'])].index
all_data.loc[index, ['birth']] = '1976'

In [65]:
all_data = all_data.groupby('user').first().reset_index()

In [66]:
## import pickle
# with open('./Data/employed_2016.csv', 'wb') as handle:
#     pickle.dump(employ_2016_users, handle, protocol=pickle.HIGHEST_PROTOCOL)


# Informative skills

In [67]:
skills = list(all_data.primary.value_counts().index)

import scipy.stats as st

#all_data = all_data[~(all_data['primary'] == '-1')].copy()
p_lehman = len(all_data[all_data.is_lehman == 1]) / len(all_data)
p_other = 1 - p_lehman
entropy_parent = - (p_lehman * np.log2(p_lehman) + p_other * np.log2(p_other))
n = len(all_data)

In [68]:
IG = []
for skill in skills:
    split = all_data[all_data.primary == skill]
    split_no = all_data[~(all_data.primary == skill)]
    
    #look at people with the skill first
#     p_split_lehman = sum(split.is_lehman)/len(split)
#     p_split_other = 1 - p_split_lehman

    #entropy_split = - (p_split_lehman * np.log2(p_split_lehman) + p_split_other * np.log2(p_split_other))
    split_lehman = sum(split.is_lehman)
    split_other = len(split) - split_lehman
    entropy_split = st.entropy([split_lehman, split_other],base=2)
    
    #look at people without the skill next
#     p_no_lehman = sum(split_no.is_lehman)/len(split_no)
#     p_no_other = 1 - p_no_lehman

    #entropy_no_split = - (p_no_lehman * np.log2(p_no_lehman) + p_no_other * np.log2(p_no_other))
    no_lehman = sum(split_no.is_lehman)
    no_other = len(split_no) - no_lehman
    entropy_no_split = st.entropy([no_lehman, no_other],base=2)
    
    #weight by number in each split
    left = len(split)
    right = len(split_no)
    entropy_children = left/n * entropy_split + right/n * entropy_no_split
    
    IG.append(entropy_parent - entropy_children)

In [69]:
sorted_ig_indices = np.flip(np.argsort(IG))

In [70]:
skills_by_ig = [skills[index] for index in sorted_ig_indices]

In [71]:
skills_by_ig

['Operations Management',
 'Insurance',
 'Business Development',
 'Product Management',
 '-1',
 'Human Resources (Senior)',
 'Real Estate',
 'Mobile Telecommunications',
 'Software Engineering',
 'Technical Product Management',
 'Non-Profit and Community',
 'Banking and Finance',
 'Administration',
 'Data Analysis',
 'Industrial Management',
 'Musical Production',
 'Web Development',
 'Accounting and Auditing',
 'Construction Management',
 'Electrical Engineering',
 'Social Media and Communications',
 'Sales Management',
 'Military',
 'Video and Film Production',
 'Sales',
 'Logistics',
 'Middle Management',
 'Web Design',
 'Manufacturing and Process Management',
 'IT Management and Support',
 'Graphic Design',
 'Digital Marketing',
 'CRM and Sales Management',
 'Energy, Oil, and Gas',
 'Retail and Fashion',
 'Human Resources (Junior)',
 'Education',
 'Public Policy',
 'Personal Coaching',
 'Hospitality',
 'Healthcare',
 'Legal',
 'Visual Design',
 'Pharmaceutical',
 'Recruiting']

# Continuing regression

In [72]:
# data deemed informative by information gain. Missing is coded as '-1'
informative_skills = skills_by_ig[0:5]
#convert uninformative skills to '0'
not_informative = ~all_data.primary.isin(informative_skills)
all_data.loc[not_informative, 'primary'] = 0
all_data['job_category'] = all_data.user.apply(lambda x: full_mapping[x])

# make sure typing is consistent for each category
X = all_data[['birth', 'gender', 'primary', 'education', 'elite']].copy()
X['education'] = X['education'].apply(str)
X['gender'] = X['gender'].apply(str)
X['birth'] = X['birth'].astype(int)
X['elite'] = X['elite'].astype(int)
X = pd.get_dummies(data=X, drop_first=True)
X = sm.tools.add_constant(X)

y = all_data['is_lehman']

# regress y on X
logit = sm.Logit(y, X)
results = logit.fit(maxiter = 100)

# get propensities
all_data['propensity'] = results.predict(X)

# Begin matching process. Map each user to its propensity
user_to_propensity = dict(zip(all_data.user, all_data.propensity))

# get lehman and non-lehman guys
lehman = all_data[all_data['is_lehman'] == 1]
non_lehman = all_data[all_data['is_lehman'] == 0]

Optimization terminated successfully.
         Current function value: 0.366821
         Iterations 8


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [73]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              is_lehman   No. Observations:                78944
Model:                          Logit   Df Residuals:                    78927
Method:                           MLE   Df Model:                           16
Date:                Mon, 20 Jan 2020   Pseudo R-squ.:                 0.01096
Time:                        09:16:00   Log-Likelihood:                -28958.
converged:                       True   LL-Null:                       -29279.
Covariance Type:            nonrobust   LLR p-value:                2.929e-126
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const                           -28.9814      3.147     -9.211      0.000     -35.148     -22.814
birth                             0.0137      0.002      8.636      0.000       0.011       0.017
elite                             0.2183      0.032      6.799      0.000       0.155       0.281
gender_1                         -0.2492      0.032     -7.734      0.000      -0.312      -0.186
gender_2                         -0.1641      0.027     -6.092      0.000      -0.217      -0.111
primary_-1                       -0.0074      0.023     -0.315      0.753      -0.053       0.039
primary_Business Development     -0.6814      0.139     -4.916      0.000      -0.953      -0.410
primary_Insurance                -1.8187      0.233     -7.808      0.000      -2.275      -1.362
primary_Operations Management    -1.0350      0.101    -10.244      0.000      -1.233      -0.837
primary_Product Management       -2.0300      0.583     -3.483      0.000      -3.172      -0.888
education_1                      -0.3046      0.115     -2.651      0.008      -0.530      -0.079
education_2                      -0.5507      0.197     -2.796      0.005      -0.937      -0.165
education_3                       0.1103      0.164      0.671      0.502      -0.212       0.432
education_4                       0.0533      0.027      1.958      0.050   -6.06e-05       0.107
education_5                      -0.0961      0.039     -2.451      0.014      -0.173      -0.019
education_6                       0.2129      0.039      5.515      0.000       0.137       0.289
education_7                       0.1906      0.063      3.027      0.002       0.067       0.314
=================================================================================================
"""

In [74]:
np.mean(X.birth)

1974.6611015403323

In [75]:
def get_closest(row):
    # return user ID of closest match with the same job title
    role = row.job_category
    score = row.propensity
    others_by_role = non_lehman[non_lehman.job_category == role].set_index('user')
    return np.absolute(others_by_role['propensity'] - score).idxmin()


# get closest match for each lehman guy
lehman['match'] = lehman.apply(get_closest, axis=1)
lehman['match_propensity'] = lehman.match.apply(lambda x: user_to_propensity[x])

lehman_matches = lehman

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [78]:
lehman[lehman.user == 'ee4460d1-2bfd-3869-be44-353e1d8edf70']

,user,birth,gender,primary,education,elite,start,end,role,company,normalized_company,ticker,industry,educational,is_lehman,job_category,propensity,match,match_propensity
73495,ee4460d1-2bfd-3869-be44-353e1d8edf70,1976,1,-1,4,False,2007-12-01,2009-01-01,"Assistant Vice President,vice president,assistant",Lehman Brothers / Barclays Capital,Lehman Brothers,LEH,522110,False,1,vp,0.114646,06f62a48-4126-326a-8f4c-8fc668c2fb80,0.114646


In [76]:
lehman['ee4460d1-2bfd-3869-be44-353e1d8edf70', :]

TypeError: '('ee4460d1-2bfd-3869-be44-353e1d8edf70', slice(None, None, None))' is an invalid key

# Naive proportion finance

In [ ]:
drop = ['length', 'gender', 'primary',
        'primary_weight', 'secondary', 'secondary_weight',
        'city', 'country', 'education', 'elite', '.', '??',
        '/', 'department', 'exchange',
        'public', 'location_company', 'degree', 'elite_education',
        'major', 'department', 'FIGI', 'last_update']

finance_data = {company_name : company_data.drop(labels=drop, axis=1) for company_name, company_data in data.items()}

def prop_finance(company_data):
    # exclude values with tickers in the categories
    copy = company_data.copy()
    users = company_data.groupby('user')
    total_users = len(users)
    # get each person's most recent job
    recent_jobs = users.first()
    # sum the people who stayed in finance industries
    stayed_finance = sum(recent_jobs['industry'].str.startswith('52', na=False))
    return stayed_finance, total_users, stayed_finance/total_users

# Apply mappings for missing industries with the dictionary, then mask to look only at job entries that start before
# '2016-1-1' and end after '2016-1-1'.
filtered_data = {company_name: filter_and_mask(company_data) for company_name, company_data in finance_data.items()}

# calculate the proportion that stayed in finance as of 2016-01-01
prop_stayed_finance = {company_name: prop_finance(company_data) for company_name, company_data in filtered_data.items()}
prop_stayed_finance

# # same as before, but don't filter the data with the dictionary
# filtered_data = {company_name: mask(company_data) for company_name, company_data in finance_data.items()}
# prop_unfiltered = {company_name: prop_finance(company_data) for company_name, company_data in filtered_data.items()}
# prop_unfiltered

# Matched proportion stayed in finance

In [ ]:
lehman_to_match = dict(zip(lehman_matches.user, lehman_matches.match))

lehman = filtered_data['leh'].copy()
lehman['match'] = lehman.user.apply(lambda x : lehman_to_match[x])

matched_users = list(lehman.match.unique())

non_lehman =  pd.concat([filtered_data['db'], filtered_data['gs'], filtered_data['ms'], filtered_data['ubs']])
matches = non_lehman[non_lehman.user.isin(matched_users)]

matches = matches.groupby('user').first().reset_index()
matches['stayed_finance'] = matches['industry'].str.startswith('52', na=False)

In [ ]:
match_to_stayed_finance = dict(zip(matches.user, matches.stayed_finance))
match_to_missing_industry = dict(zip(matches.user, pd.isnull(matches.industry)))

In [ ]:
# from collections import defaultdict

# match_to_stayed_finance = defaultdict(lambda: None, match_to_stayed_finance)

In [ ]:
lehman_most_recent = lehman.copy()

In [ ]:
lehman_most_recent = lehman.groupby('user').first()
lehman_most_recent['match_stayed_finance'] = lehman_most_recent.match.apply(lambda x: match_to_stayed_finance[x])
lehman_most_recent['lehman_stayed_finance'] = lehman_most_recent.industry.str.startswith('52', na = False)
lehman_most_recent['job_category'] = lehman_most_recent.index.to_series().apply(lambda x: full_mapping[x])

In [ ]:
roles = lehman_most_recent.job_category.unique()
stayed_finance = {'role' : [], 'lehman_stayed' : [], 'match_stayed' : [], 'total' : [], 'proportion_lehman':[], 'proportion_match' :[], 'zscore': []}

for role in roles:
    role_data = lehman_most_recent[lehman_most_recent.job_category == role]
    lehman_stayed = sum(role_data.industry.str.startswith('52', na = False))
    match_stayed = sum(role_data.match_stayed_finance)
    total = len(role_data)
    prop_lehman = lehman_stayed/total
    prop_match = match_stayed/total
    zscore = (prop_lehman - prop_match) / (prop_lehman * prop_match *(2/total))**(1/2)
    stayed_finance['role'].append(role)
    stayed_finance['lehman_stayed'].append(lehman_stayed)
    stayed_finance['match_stayed'].append(match_stayed)
    stayed_finance['total'].append(total)
    stayed_finance['proportion_lehman'].append(prop_lehman)
    stayed_finance['proportion_match'].append(prop_match)
    stayed_finance['zscore'].append(zscore)
role_data = lehman_most_recent[lehman_most_recent.job_category == role]

lehman_stayed = sum(lehman_most_recent.industry.str.startswith('52', na = False))
match_stayed = sum(lehman_most_recent.match_stayed_finance)
total = len(lehman_most_recent)
prop_lehman = lehman_stayed/total
prop_match = match_stayed/total
zscore = (prop_lehman - prop_match) / (prop_lehman * prop_match *(2/total))**(1/2)

stayed_finance['role'].append('all_roles')
stayed_finance['lehman_stayed'].append(lehman_stayed)
stayed_finance['match_stayed'].append(match_stayed)
stayed_finance['total'].append(total)
stayed_finance['proportion_lehman'].append(lehman_stayed/total)
stayed_finance['proportion_match'].append(match_stayed/total)
stayed_finance['zscore'].append(zscore)

In [ ]:
df = pd.DataFrame(stayed_finance)

df = df[df.total>= 200]

df

toPlot = df[['role', 'proportion_lehman', 'proportion_match']].set_index('role').stack().reset_index()
toPlot = toPlot.rename({'level_1' : 'company', 0 : 'proportion'}, axis = 1)

from matplotlib import pyplot
import seaborn as sns
%matplotlib inline

sns.set(rc={'figure.figsize':(20,5)})

sns.lineplot( x = 'role', y = 'proportion', hue = 'company', data = toPlot);

In [ ]:
df.sort_values(by = 'role').to_csv('./Deliverables/proportion_finance.csv')

# Missing industries Lehman vs match

In [ ]:
lehman_most_recent['lehman_missing_industry'] = pd.isnull(lehman_most_recent.industry)
lehman_most_recent['match_missing_industry'] = lehman_most_recent.match.apply(lambda x : match_to_missing_industry[x])

In [ ]:
len(lehman_most_recent)

In [ ]:
(sum(lehman_most_recent.match_missing_industry), sum(lehman_most_recent.lehman_missing_industry))

In [ ]:
lehman_guys = lehman_most_recent[['lehman_stayed_finance']].copy().rename({'lehman_stayed_finance' : 'stayed_finance'}, axis = 1)

lehman_guys['is_lehman'] = 1

lehman_guys

non_lehman_guys = lehman_most_recent[['match_stayed_finance']].copy().rename({'match_stayed_finance' : 'stayed_finance'}, axis = 1)
non_lehman_guys['is_lehman'] = 0

all_data = pd.concat([lehman_guys,non_lehman_guys])

all_data

X = all_data['is_lehman']
y = all_data['stayed_finance']


model = sm.Probit(y, X)
results = model.fit()

results.get_margeff().summary()

# Breaks

In [ ]:
def prop_breaks(company_name, company_data):
    """
    Outputs proportion of "TIME_OFF" or "MISSING" entries in a
    dataset as the tuple (numerator, denominator).
    """

    def get_breaks():
        """
        Outputs the number of employees that have any "TIME_OFF" or "MISSING" entries after 2008.
        """
        date_2008 = pd.to_datetime('2008-01-01')
        # look only at data after 2008
        after_2008 = company_data[company_data['start'] > date_2008]
        # groupby user, aggregate by looking at the ticker and seeing if the person has had any time off
        return sum(after_2008.groupby('user').ticker.agg(lambda x: any((x == 'TIME_OFF') | (x == 'MISSING'))))
    
    num_company_users = len(company_data.groupby('user'))
    num_breaks = get_breaks()
    return company_name, num_breaks, num_company_users, num_breaks/num_company_users

drop = ['length', 'gender', 'primary',
        'primary_weight', 'secondary', 'secondary_weight',
        'city', 'country', 'education', 'elite', '.', '??',
        '/', 'department', 'exchange',
        'public', 'location_company', 'educational', 'degree', 'elite_education',
        'major', 'department', 'FIGI', 'last_update', 'name', 'birth','end', 'role',
        'company','normalized_company','industry']

breaks_data = {company_name : company_data.drop(labels=drop, axis=1) for company_name, company_data in data.items()}

lehman = breaks_data['leh'].copy()
date_2008 = pd.to_datetime('2008-01-01')
# look only at data after 2008
lehman = lehman[lehman['start'] > date_2008]
lehman = lehman[lehman.user.isin(list(lehman_to_match.keys()))]

lehman['match'] = lehman.user.apply(lambda x : lehman_to_match[x])
prop_breaks('leh', lehman)

In [ ]:
matches = list(lehman.match.unique())

non_lehman =  pd.concat([breaks_data['db'], breaks_data['gs'], breaks_data['ms'], breaks_data['ubs']])

non_lehman_matches = non_lehman[non_lehman.user.isin(matches)]

date_2008 = pd.to_datetime('2008-01-01')
# look only at data after 2008
after_2008 = non_lehman_matches[non_lehman_matches['start'] > date_2008]
# groupby user, aggregate by looking at the ticker and seeing if the person has had any time off
y = after_2008.groupby('user').ticker.agg(lambda x: any((x == 'TIME_OFF') | (x == 'MISSING')))

w = y.to_frame()

r = w[w.ticker == True]

r = list(r.index)

In [ ]:
non_lehman_matches['took_break'] = False

non_lehman_matches.loc[non_lehman_matches.user.isin(r), 'took_break'] = True

user_to_break = dict(zip(non_lehman_matches.user, non_lehman_matches.took_break))

x = lehman.groupby('user').first()

x['break'] = x.match.apply(lambda x: user_to_break[x])

('other', sum(x['break']), len(x), sum(x['break'])/len(x))

In [ ]:
lehman_guys = lehman.groupby('user').first()
lehman_guys['break'] = lehman.groupby('user').ticker.agg(lambda x: any((x == 'TIME_OFF') | (x == 'MISSING'))).copy()

In [ ]:
lehman_guys['is_lehman'] = 1
lehman_guys = lehman_guys[['break', 'is_lehman']]

In [ ]:
non_lehman_guys = x[['break']].copy()
non_lehman_guys['is_lehman'] = 0

In [ ]:
all_data = pd.concat([lehman_guys,non_lehman_guys])

all_data

X = all_data['is_lehman']
y = all_data['break']


model = sm.Probit(y, X)
results = model.fit()

results.get_margeff().summary()